# **Import Libraries**

In [ ]:
import os
import gc
import sys
import time
import shutil

import random
import pickle

from ast import literal_eval
from tqdm import tqdm as print_progress
from glob import glob

import dask.dataframe as dd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
os.environ['TF_KERAS'] = '1'

import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
from tensorflow.keras.layers import (
    Layer, 
    Input, InputLayer, Embedding, 
    Dropout, Dense, 
    Dot, Concatenate, Average, Add,
    Bidirectional, LSTM,
    Lambda, Reshape
)
from tensorflow.keras.activations import softmax, sigmoid
from tensorflow.keras.initializers import Identity, GlorotNormal
from tensorflow.keras.utils import plot_model

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
pip install stellargraph

In [ ]:
pip install gradient-centralization-tf

# **Load data**

In [ ]:
label_encoder = pickle.load(open('../input/hotel-comment/label_encoder.pkl', 'rb'))
labels = list(label_encoder.classes_)
len(labels)

# **Data Generator**

In [ ]:
labels_matrix = np.load('../input/hotel-comment-valtest-distiluse/labels_embeddings.npy')
labels_matrix.shape

In [ ]:
import sklearn
from ast import literal_eval
from tensorflow.keras.utils import Sequence, to_categorical


class DataGenerator(Sequence):

    def __init__(self,
                 data_root,
                 labels_fixed: np.array,
                 max_seq_len: int=512,
                 batch_size: int = 256, 
                 shuffle: bool = True):
        
        self.data_root = data_root
        self.num_labels = labels_fixed.shape[-2]
        self.max_seq_len = max_seq_len
        self.labels_fixed = labels_fixed
        self.embedding_dim = labels_fixed.shape[-1]
        
        # list of files containing both word-embeddings and multi-labels
        if isinstance(self.data_root, str):
            self.files = glob(os.path.join(self.data_root, 'sample_*.npz'))
        elif isinstance(self.data_root, (list, tuple)):
            self.files = []
            for data_dir in self.data_root:
                self.files += glob(os.path.join(data_dir, 'sample_*.npz'))
                
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.indices = np.array(list(range(len(self.files))))
        self.on_epoch_end()

    def __len__(self):
        """
        Denotes the number of batches per epoch
        """
        return int(len(self.files) // self.batch_size)

    def __getitem__(self, index):
        """
        Generate one batch of data
        """
        # Generate indexes of the batch
        start_index = self.batch_size * index
        end_index = self.batch_size * (index+1)
        indices = self.indices[start_index:end_index]

        # Generate data
        wb_batch = []
        mtl_batch = []
        for idx in indices:
            
            sample_file = self.files[idx]
            
            # Load word embeddings
            wb_pad = np.zeros((self.max_seq_len, self.embedding_dim))
            wb = np.load(sample_file)['emb']
            wb_pad[:wb.shape[0],:] = wb
            wb_batch += [wb_pad]
            
            # Load multi-labels
            mtl = np.load(sample_file)['mtl']
            mtl_batch += [self.smooth_labels(mtl)]
        return [np.array(wb_batch), self.labels_fixed], np.array(mtl_batch)

    def smooth_labels(self, labels, factor=0.1):
        # smooth the labels
        labels *= (1 - factor)
        labels += (factor / labels.shape[-1])
        return labels

    def on_epoch_end(self):
        """
        Update indices after each epoch
        """
        if self.shuffle:
            self.indices = sklearn.utils.shuffle(self.indices)

In [ ]:
dset_paths = [
    ['../input/hotel-comment-distiluse-trainset-part-0/trainingdata', 
     '../input/hotel-comment-distiluse-trainset-part-1/trainingdata',
     '../input/hotel-comment-distiluse-trainset-part-2/trainingdata'],
    ['../input/hotel-comment-valtest-distiluse/valuatingdata'],
    ['../input/hotel-comment-testset-distiluse/testingdata']
]
data_generator = dict()
for dataset, dset_path in zip(['training', 'valuating', 'testing'], dset_paths):
    data_generator[dataset] = DataGenerator(data_root=dset_path, 
                                            labels_fixed=labels_matrix, 
                                            batch_size=128, 
                                            shuffle=True if dataset=='training' else False)

In [ ]:
len(data_generator['training']), len(data_generator['valuating'])

In [ ]:
X = data_generator['training'][0]
print(X[0][0].shape)
print(X[0][1].shape)
print(X[1].shape)

# **Load Model**

In [ ]:
class CyclicLR(Callback):
    """
    This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with some constant frequency, 
        as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or per-cycle basis.
    
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "halving":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exponential":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.

    For more detail, please read the paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {original, halving, exponential}.
            Default 'original'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """
    def __init__(self, base_lr=0.001, max_lr=0.1, step_size=2000., mode='original',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'halving':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exponential':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
            else:
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None, new_step_size=None):
        """
        Resets cycle iterations.
            Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        new_lr = self.clr()
        K.set_value(self.model.optimizer.lr, new_lr)

In [ ]:
class Adjacency(Layer):

    def __init__(self, nodes=1, weights=None, init_method='identity'):
        super(Adjacency, self).__init__()

        self.shape = (1, nodes, nodes)

        if weights is not None:
            assert weights.shape==(nodes, nodes), \
                f'Adjacency Matrix must have shape ({nodes}, {nodes})' + \
                f' while its shape is {weights.shape}'
            w_init = tf.convert_to_tensor(weights)
        else:
            init_method = init_method.lower()
            if init_method == 'identity':
                initializer = tf.initializers.Identity()
            elif init_method in ['xavier', 'glorot']:
                initializer = tf.initializers.GlorotNormal()
            w_init = initializer(shape=(nodes, nodes))

        self.w = tf.Variable(
            initial_value=tf.expand_dims(w_init, axis=0), 
            dtype="float32", trainable=True
        )

    def call(self, inputs):
        return tf.convert_to_tensor(self.w)

    def compute_output_shape(self):
        return self.shape

In [ ]:
import gctf
from stellargraph.layer import GraphAttention
from stellargraph.utils import plot_history


def buil_MAGNET(n_labels,
                embedding_dim: int,
                sequence_length: int=512, 
                lstm_units: int=64,
                dropout_rates=[0.5, 0.3],
                attention_heads=[4, 2],
                adjacency_matrix=None,
                adjacency_generation='xavier', # 'identity' or 'xavier' or 'glorot'
                feed_text_embeddings=True, # if False, add additional Embedding layer
                text_embeddings_matrix=None, # initialized weights for text Embedding layer
                feed_label_embeddings=True, # if False, add additional Embedding layer
                label_embeddings_matrix=None, # initialized weights for label Embedding layer
                ) -> Model:

    if isinstance(attention_heads, int):
        attention_heads = [attention_heads, attention_heads]
    if not isinstance(attention_heads, (list, tuple)):
        raise ValueError('`attention_heads` must be INT, LIST or TUPLE')

    # 1. Sentence Representation
    if feed_text_embeddings:
        sentence_model = Sequential(name='sentence_model')
        sentence_model.add(Dropout(dropout_rates[0], input_shape=(sequence_length, embedding_dim), name='word_embeddings'))
        word_inputs, word_embeddings = sentence_model.inputs, sentence_model.outputs
    else:
        word_inputs = Input(shape=(sequence_length, ), name='word_inputs')
        embedding_args = {
            'input_dim': sequence_length,
            'output_dim': embedding_dim,
            'name': 'word_embeddings'
        }
        if text_embeddings_matrix is not None \
            and text_embeddings_matrix.shape==(sequence_length, embedding_dim):
            embedding_args['weights'] = [text_embeddings_matrix]
        word_embeddings = Embedding(**embedding_args)(word_inputs)
        word_embeddings = Dropout(dropout_rates[0], name='WE_dropout')(word_embeddings)
    
    forward_rnn = LSTM(units=lstm_units, return_sequences=True, name='forward_rnn')
    backward_rnn = LSTM(units=lstm_units, return_sequences=True, name='backward_rnn', go_backwards=True)
    bidir_rnn = Bidirectional(layer=forward_rnn, backward_layer=backward_rnn, merge_mode="concat", name='bidir_rnn')
    
    sentence_repr = bidir_rnn(word_embeddings)
    sentence_repr = K.mean(sentence_repr, axis=1)
    # print(f"sentence_repr: {K.int_shape(sentence_repr)}")

    # 2. Labels Representation
    if feed_label_embeddings:
        label_inputs = Input(batch_shape=(1, n_labels, embedding_dim), name='label_embeddings')
        label_embeddings = label_inputs
    else:
        label_inputs = Input(batch_shape=(1, n_labels), name='label_inputs')
        embedding_args = {'input_dim': n_labels,
                          'output_dim': embedding_dim,
                          'name': 'label_embeddings'}
        if label_embeddings_matrix is not None \
            and label_embeddings_matrix.shape==(n_labels, embedding_dim):
            embedding_args['weights'] = [label_embeddings_matrix]
        label_embeddings = Embedding(**embedding_args)(label_inputs)
        label_embeddings = Dropout(rate=dropout_rates[0], name='LE_dropout')(label_embeddings)
    label_embeddings = Dense(units=embedding_dim//2, name='label_embeddings_reduced')(label_embeddings)
    # print(f"label_inputs: {K.int_shape(label_inputs)}")

    label_correlation = Adjacency(nodes=n_labels, 
                                  weights=adjacency_matrix,
                                  init_method=adjacency_generation)(label_embeddings)
    # print(f"label_correlation: {K.int_shape(label_correlation)}")

    label_attention = GraphAttention(units=embedding_dim//2//attention_heads[0],
                                     activation='tanh',
                                     attn_heads=attention_heads[0],
                                     in_dropout_rate=dropout_rates[1],
                                     attn_dropout_rate=dropout_rates[1], )([label_embeddings, label_correlation])
    # print(f"label_attention: {K.int_shape(label_attention)}")

    label_residual = Add(name='label_residual')([label_attention, label_embeddings])
    # print(f"label_residual: {K.int_shape(label_residual)}")

    label_repr = GraphAttention(units=2*lstm_units,
                                activation='tanh',
                                attn_heads_reduction='average',
                                attn_heads=attention_heads[1],
                                in_dropout_rate=dropout_rates[1],
                                attn_dropout_rate=dropout_rates[1], )([label_residual, label_correlation])

    label_repr = K.sum(label_repr, axis=0, keepdims=False)
    # print(f"label_repr: {K.int_shape(label_repr)}")

    # 3. Prediction
    prediction = tf.einsum('Bk,Nk->BN', sentence_repr, label_repr)
    prediction = sigmoid(prediction)
    # print(f"prediction: {K.int_shape(prediction)}")

    return Model(inputs=[word_inputs, label_inputs], outputs=prediction, name='MAGNET')

In [ ]:
def noam_scheme(global_step, init_lr, warmup_steps=16):
    """
    Noam scheme learning rate decay
        init_lr: (scalar) initial learning rate. 
        global_step: (scalar) current training step
        warmup_steps: (scalar) During warmup_steps, learning rate increases until it reaches init_lr.
    """
    step = tf.cast(global_step+1, dtype=tf.float32, name="global_step")
    return init_lr * (warmup_steps**0.5) * tf.minimum(step*(warmup_steps**-1.5), step**-0.5)

In [ ]:
def weighted_cross_entropy(y_true, y_pred, pos_weight=1.69):
    losses = y_true * -K.log(y_pred) * pos_weight + (1-y_true) * -K.log(1-y_pred)
    losses = K.clip(losses, 0.0, 11.27)
    return K.mean(losses)

In [ ]:
def categorical_focal_loss(n_classes, alpha: float=0.25, gamma: float=2.):
    alpha = np.array([[alpha]*n_classes], dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.-epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1-y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed

In [ ]:
import gctf
from stellargraph.utils import plot_history


class MAGNET:

    def __init__(self, n_labels: int, embedding_dim: int):

        self.embedding_dim = embedding_dim

        # Build model(s)
        print(f"\n\n\nBuilding MAGNET ...\n\n\n")
        self.model = buil_MAGNET(n_labels, embedding_dim=embedding_dim, sequence_length=512, lstm_units=32)
        self.model.summary()

    def compile(self, model_saved: str, logs_path: str, schedule_step: int, verbose: int=1):
                    
        # Compile optimizer, loss & metric functions
        print(f"Compiling MAGNET using \n\tgrad-centralized ADAM, \n\ttop-k Accuracy, \n\tweighted Cross-Entropy \n...")
        self.model.compile(optimizer=gctf.optimizers.adam(learning_rate=0.00169), 
                           # optimizer=Adam(learning_rate=0.001), 
                           metrics=["accuracy", TopKCategoricalAccuracy(k=3)],
                           loss=categorical_crossentropy)

        # Define Callbacks
        return [
            # TensorBoard(log_dir=logs_path),
            # ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=verbose),
            CyclicLR(mode='exponential', base_lr=1e-7, max_lr=1e-3, step_size=schedule_step),
            ModelCheckpoint(filepath=model_saved, monitor='accuracy', save_weights_only=True, save_best_only=False, save_freq='epoch'),
            # LearningRateScheduler(noam_scheme),
            # EarlyStopping(monitor='val_accuracy', mode='max', restore_best_weights=True, min_delta=1e-7, patience=7, verbose=verbose),
        ]

    def finetune(self, train_generator, val_generator, model_saved: str, logs_path: str, n_loops: int=3, verbose: int=1):
        # Compile
        schedule_step = len(train_generator) // 2
        custom_callbacks = self.compile(model_saved, logs_path, schedule_step, verbose)

        # Define part(s) of layers for fine-tuning
        graph_layers = ['adjacency', 'graph_attention', 'graph_attention_1']
        word_layers = ['bidir_rnn', 'label_embeddings_reduced']
        train_histories = []

        ######################################
        #             FINE-TUNING            #
        ######################################

        print(f"[Fine-tuning MAGNET]")
        train_args = {
            'generator': train_generator,
            'steps_per_epoch': len(train_generator)//3,
            'validation_data': val_generator,
            'validation_steps': len(val_generator),
            'callbacks': custom_callbacks
        }
        for l in range(n_loops):
            
            print(f"Training loop {l+1}")

            # Step 1: Train ALL layers
            for layer in self.model.layers:
                layer.trainable = True

            print(f"\tStep 1: Training ALL layers ...")
            train_history = self.model.fit_generator(initial_epoch=l*5, epochs=l*5+2, **train_args)
            train_histories.append(train_history)

            # Step 2: Train GRAPH layers
            for layer in self.model.layers:
                layer.trainable = True if layer.name in graph_layers else False

            print(f"\tStep 2: Training GRAPH layers ...")
            train_history = self.model.fit_generator(initial_epoch=l*5+2, epochs=l*5+3, **train_args)
            train_histories.append(train_history)

            # Step 3: Train EMBEDDING layers
            for layer in self.model.layers:
                layer.trainable = True if layer.name in word_layers else False

            print(f"\tStep 3: Training EMBEDDING layers ...")
            train_history = self.model.fit_generator(initial_epoch=l*5+3, epochs=l*5+4, **train_args)
            train_histories.append(train_history)

            # Step 4: Train ALL layers
            for layer in self.model.layers:
                layer.trainable = True

            print(f"\tStep 4: Training ALL layers ...")
            train_history = self.model.fit_generator(initial_epoch=l*5+4, epochs=l*5+5, **train_args)
            train_histories.append(train_history)

            # Reduce learning rate
            # custom_callbacks[0].base_lr /= 1.69
            # custom_callbacks[0].max_lr /= 1.69

        return train_histories

    def train(self, train_generator, val_generator, 
                    model_saved: str, logs_path: str,
                    max_epochs: int=50, verbose: int=1):
        # Compile
        schedule_step = len(train_generator) // 2
        custom_callbacks = self.compile(model_saved, logs_path, schedule_step, verbose)

        # Training
        train_history = self.model.fit_generator(generator=train_generator,
                                                 steps_per_epoch=len(train_generator),
                                                 validation_data=val_generator,
                                                 validation_steps=len(val_generator),
                                                 callbacks=custom_callbacks, 
                                                 epochs=max_epochs,
                                                 initial_epoch=0)
        return train_history

    def load_weights(self, weight_path: str):
        self.model.load_weights(weight_path)

    def predict(self, label_embeddings: np.array, sent_embeddings: np.array):
        sent_embeddings = np.reshape(sent_embeddings, (1, 512, self.embedding_dim))
        preds = self.model.predict([sent_embeddings, label_embeddings]).tolist()
        return preds[0]

In [ ]:
N_LABELS = labels_matrix.shape[-2]
embedding_dim = labels_matrix.shape[-1]

model = MAGNET(n_labels=N_LABELS, embedding_dim=embedding_dim)

# **Train**

In [ ]:
models_path = '/kaggle/working/models'
if not os.path.isdir(models_path):
    os.makedirs(models_path)

logs_path = '/kaggle/working/logs'
if not os.path.isdir(logs_path):
    os.makedirs(logs_path)
    
pred_dir = '/kaggle/working/predictions'
if not os.path.isdir(pred_dir):
    os.makedirs(pred_dir)
    
model_format = 'ep={epoch:03d}_acc={accuracy:.3f}_val_acc={val_accuracy:.3f}_topk={top_k_categorical_accuracy:.3f}_val_topk={val_top_k_categorical_accuracy:.3f}.h5'
model_saved =  os.path.join(models_path, model_format)

In [ ]:
train_histories = model.finetune(data_generator['training'], 
                                 data_generator['valuating'], 
                                 model_saved=model_saved, 
                                 logs_path=logs_path, 
                                 n_loops=2, 
                                 verbose=1)

In [ ]:
for t_i, train_history in enumerate(train_histories):
    with open(f'train_history_{t_i}.hst', 'wb') as f_writer:
        pickle.dump(train_history.history, f_writer)

In [ ]:
os.chdir(r'/kaggle/working')
dir_path = '/kaggle/working/'
shutil.make_archive(dir_path+"data", 'zip', dir_path)